In [ ]:
import requests
from PIL import Image
from io import BytesIO
import os
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon
from google.colab import drive

In [ ]:
# Constants for calculating the step size
EARTH_CIRCUMFERENCE = 40075017  # Earth's circumference in meters
TILE_SIZE = 640  # Google Static Map tile size in pixels
ZOOM_LEVEL = 20  # Zoom level 20


In [ ]:
# Function to calculate meters per pixel at a given latitude
def meters_per_pixel(latitude, zoom):
    return (EARTH_CIRCUMFERENCE * math.cos(math.radians(latitude))) / (2 ** (zoom + 8))


In [ ]:
# Calculate the approximate size (in degrees) covered by each tile at Zoom 20
def tile_step_size(latitude, zoom):
    meters_per_tile = TILE_SIZE * meters_per_pixel(latitude, zoom)
    # Convert meters to degrees latitude (1 degree = ~111.32 km)
    lat_step = meters_per_tile / 111320
    # Convert meters to degrees longitude (adjusted by latitude)
    lon_step = meters_per_tile / (111320 * math.cos(math.radians(latitude)))
    return lat_step, lon_step

In [ ]:
# Google Static Maps API base information
api_key = ''  # Replace with your Google Static Maps API key
base_url = "https://maps.googleapis.com/maps/api/staticmap?"

# Load GeoJSON file
gdf = gpd.read_file('/content/drive/MyDrive/Kamakura.geojson')# Replace with the path to your GeoJSON file from the predict city
polygon = gdf.geometry[0]  # Assuming there's only one polygon for the city boundary

# Save directory in Google Drive
save_dir = ''# Replace with the path to your save directory
os.makedirs(save_dir, exist_ok=True)


In [ ]:
# Get the bounding box for the polygon
minx, miny, maxx, maxy = polygon.bounds

# Start scanning from the southwest corner
lat = miny
while lat < maxy:
    lon = minx
    lat_step, lon_step = tile_step_size(lat, ZOOM_LEVEL)  # Calculate dynamic step size for the current latitude
    while lon < maxx:
        # Check if the point is inside the polygon
        point = Point(lon, lat)
        if polygon.contains(point):
            # Define the center point for the current API request
            center = f"{lat},{lon}"
            params = {
                'center': center,
                'zoom': ZOOM_LEVEL,
                'size': f"{TILE_SIZE}x{TILE_SIZE}",
                'maptype': 'satellite',
                'key': api_key,
                'heading': 0,  # Align the map to true north
                'pitch': 0     # Ensure a vertical top-down view
            }

            # Send the API request
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                # Load the image from the response content
                img = Image.open(BytesIO(response.content))

                # Save the image to Google Drive
                img_save_path = os.path.join(save_dir, f"map_{lat}_{lon}.png")
                img.save(img_save_path)
                print(f"Image saved: {img_save_path}")
            else:
                print(f"Request failed with status code: {response.status_code}, location: {lat},{lon}")

        # Move to the next longitude (right)
        lon += lon_step

    # Move to the next latitude (up)
    lat += lat_step

print("All images successfully downloaded.")

Streaming output truncated to the last 5000 lines.
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.55726623723066.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.55812454019605.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.55898284316143.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.55984114612681.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.5606994490922.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.56155775205758.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.56241605502296.png
Image saved: /content/drive/MyDrive/solar_PV_prediction/Japan/Kamakura/map_35.31691064343482_139.56327435798835.png
Image saved: /content/

Truncate the Predict images from Japan

In [ ]:
import os
from PIL import Image

# Define the input and output directories
pre_image_dir = ''# Replace with the path to your save the import image directory
output_dir = ""# Replace with the path to your save the output image direct (after truncate)

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List all files in the input directory
all_images = os.listdir(pre_image_dir)
print(f"Processing images in {pre_image_dir}: {all_images}")

# Process each image in the directory
for image in all_images:
    # Only process PNG files
    if not image.endswith(".png"):
        print(f"Skipping {image} as it is not a PNG file.")
        continue

    # Open the image
    img_path = os.path.join(pre_image_dir, image)
    img = Image.open(img_path)
    print(f"Opened image: {img_path}")

    # Split the image into four parts
    width, height = img.size
    mid_x = width // 2
    mid_y = height // 2

    img1 = img.crop((0, 0, mid_x, mid_y))
    img2 = img.crop((mid_x, 0, width, mid_y))
    img3 = img.crop((0, mid_y, mid_x, height))
    img4 = img.crop((mid_x, mid_y, width, height))

    # Get the base filename and extension
    base_name, ext = os.path.splitext(image)

    # Save the cropped images in the output directory
    img1.save(os.path.join(output_dir, f"{base_name}_1{ext}"))
    img2.save(os.path.join(output_dir, f"{base_name}_2{ext}"))
    img3.save(os.path.join(output_dir, f"{base_name}_3{ext}"))
    img4.save(os.path.join(output_dir, f"{base_name}_4{ext}"))

    print(f"Saved cropped images for {image} in {output_dir}")
